# First exploration of selected files

In [4]:
import pandas as pd
import numpy as np

We obtain the csv file with our categorization of the files in our dataset.

In [7]:
file_categorization = pd.read_csv("file-categorization.csv", sep = ';')

In [16]:
by_disorder_countries = {}
not_disorder_specific_countries = {}
survey_countries = {}

for i in range (len(file_categorization)):
    row = file_categorization.iloc[i]
    filename, category = row['filename'], row['category']
    if not (category == "unused" or category == "unsure"):
        if category == "by-disorder":
            df = pd.read_csv("mental-health-dataset/" + filename)
            by_disorder_countries[filename] = sorted(list(set(list(df['Entity']))))
        elif category == "not-disorder-specific":
            not_disorder_specific_countries[filename] = sorted(list(set(list(df['Entity']))))
        elif category == "survey":
            survey_countries[filename] = sorted(list(set(list(df['Entity']))))

### Checking for cohesive country list in the category `by-disorder`

In [18]:
for file in by_disorder_countries:
    print(by_disorder_countries[file])

['Afghanistan', 'African Region (WHO)', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Democratic Republic of Congo', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'East Asia & Pacific (WB)', 'East Timor', 'Eastern Mediterranean Region (WHO)', 'Ecuador', 'Egypt', 'El Salvador', 'England', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Europe & Central Asia (WB)', 'European Region (WHO)', 'Fiji', 'Finland', 'France', 'G20', 'G

Our lists of countries do not match, meaning that we do not have all of the information for all of the countries (we'll have gaps/holes).

### Checking for cohesive country list in the category `not-disorder-specific`

In [17]:
for file in not_disorder_specific_countries:
    print(not_disorder_specific_countries[file])

['Abkhazia', 'Afghanistan', 'Africa', 'Africa (IHME GBD)', 'Africa (UN)', 'Akrotiri and Dhekelia', 'Aland Islands', 'Albania', 'Algeria', 'America (IHME GBD)', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Asia', 'Asia (IHME GBD)', 'Asia (UN)', 'Australia', 'Austria', 'Austria-Hungary', 'Azerbaijan', 'Baden', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Bavaria', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Channel Islands', 'Chile', 'China', 'Christmas Island', 'Cocos Islands', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba'

Same lists!

### Checking for cohesive country list in the category `survey`

In [19]:
for file in survey_countries:
    print(survey_countries[file])

['Abkhazia', 'Afghanistan', 'Africa', 'Africa (IHME GBD)', 'Africa (UN)', 'Akrotiri and Dhekelia', 'Aland Islands', 'Albania', 'Algeria', 'America (IHME GBD)', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Asia', 'Asia (IHME GBD)', 'Asia (UN)', 'Australia', 'Austria', 'Austria-Hungary', 'Azerbaijan', 'Baden', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Bavaria', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Channel Islands', 'Chile', 'China', 'Christmas Island', 'Cocos Islands', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba'

Same lists!